<h1>1. Duomenų nuskaitymas.</h1>

Pirmam eksperimentui nuskaitomi Enron1 duomenys. Tai .txt formate išsaugotų laiškų rinkiniai, padalinti į gerus laiškus (ham) ir šlamšto laiškus (spam).

In [ ]:
import glob2 #duomenų nuskaitymui
import pandas as pd #duomenų formatavimui
import numpy as np #skaičiavimams ir formatavimui
from nltk.corpus import stopwords #angliškų nereikšminių žodžių rinkinys
from sklearn import metrics #(galbūt) reikiamos metrikos
from sklearn.feature_extraction.text import CountVectorizer #duomenų formatavimui modeliams
from sklearn.naive_bayes import MultinomialNB #bajeso modelis
from sklearn.neighbors import KNeighborsClassifier #knn modelis
from sklearn.svm import SVC #svc modelis?
from sklearn.model_selection import cross_val_score #kryžminė patikra modelių optimizavimui
from sklearn.pipeline import make_pipeline #?
from sklearn.preprocessing import StandardScaler #?

In [ ]:
duomenys = pd.DataFrame(columns = ("ID", "Tipas", "Tekstas"))
i = 0

for failas in glob2.glob("./enron1/ham/*.txt"):
    try:
        with open(failas, encoding = "utf-8") as atdaras_failas:
            failo_tekstas = atdaras_failas.read().replace("\n", " ")
            i += 1
            eilute = pd.DataFrame(data = [[i, "ham", failo_tekstas]], columns = ("ID", "Tipas", "Tekstas"))
            duomenys = pd.concat([duomenys, eilute], axis = 0)
    except:
        None

for failas in glob2.glob("./enron1/spam/*.txt"):
    vardas = failas
    try:
        with open(vardas, encoding = "utf-8") as atdaras_failas:
            failo_tekstas = atdaras_failas.read().replace("\n", " ")
            i += 1
            eilute = pd.DataFrame(data = [[i, "spam", failo_tekstas]], columns = ("ID", "Tipas", "Tekstas"))
            duomenys = pd.concat([duomenys, eilute], axis = 0)
    except:
        None
        

In [ ]:
print(duomenys.shape)
print(len(glob2.glob("./enron1/ham/*.txt")))
print(len(glob2.glob("./enron1/spam/*.txt")))
print(duomenys["Tipas"].value_counts(normalize = True))
print(duomenys["Tipas"].value_counts(normalize = False))

<h1>2. Duomenų paskirstymas į treniravimo ir testavimo rinkinius, ir apdorojimas.</h1>

Duomenys paskirstomi į treniravimo ir testavimo duomenis su 70%/30% santykiu. Iš jų pašalinami skirybos ženklai, suvienodinamos raidės. 

In [ ]:
maisyti_duomenys = duomenys.sample(frac = 1, random_state = 333)

indeksas = round(len(maisyti_duomenys)*0.7)

treniravimo_duomenys = maisyti_duomenys[:indeksas].reset_index(drop = True)
testavimo_duomenys = maisyti_duomenys[indeksas:].reset_index(drop = True)

print(treniravimo_duomenys.shape)
print(testavimo_duomenys.shape)

nereiksminiai_z = stopwords.words("english")

In [ ]:
def apdoroti_tekstus(ivestas_tekstas):
    ivestas_tekstas = ivestas_tekstas.str.replace('\W', ' ') #pašalinami skirybos ženklai
    ivestas_tekstas = ivestas_tekstas.str.replace('\n', ' ') #pašalinami newline
    ivestas_tekstas = ivestas_tekstas.str.lower() #konvertuojama į visas mažasias raides
    ivesti_zodziai = ivestas_tekstas.str.split() #tekstas suskaidomas į pavienių žodžių rinkinius
    #ivesti_zodziai = nltk.word_tokenize(ivestas_tekstas)
    
    #išmetami skaičiai ir nereikšminiai žodžiai
    tekstai_ = []
    for tekstas in ivesti_zodziai:
        tekstas_ = []
        tekstas_ = [zodis for zodis in tekstas if not zodis.isnumeric() and zodis not in nereiksminiai_z]
        tekstai_.append(tekstas_)

    return tekstai_

In [ ]:
def apdoroti_teksta(ivestas_tekstas):
    ivestas_tekstas = ivestas_tekstas.replace('\W', ' ') #pašalinami skirybos ženklai
    ivestas_tekstas = ivestas_tekstas.replace('\n', ' ') #pašalinami newline
    ivestas_tekstas = ivestas_tekstas.lower() #konvertuojama į visas mažasias raides
    ivesti_zodziai = ivestas_tekstas.split() #tekstas suskaidomas į pavienių žodžių rinkinius
    #ivesti_zodziai = nltk.word_tokenize(ivestas_tekstas)
    
    #išmetami skaičiai ir nereikšminiai žodžiai
    tekstai_ = []
    for tekstas in ivesti_zodziai:
        tekstas_ = []
        tekstas_ = [zodis for zodis in tekstas if not zodis.isnumeric() and zodis not in nereiksminiai_z]
        tekstai_.append(tekstas_)

    return tekstai_

<h1>3. Žodyno sudarymas (rankiniu būdu) (vykdant kodą praleisti šitą skyrių, jis yra demonstracijai).</h1>

Iš laiškų tekstuose esančių žodžių sudaromi unikalus žodynas, kuriame yra visi skirtingi žodžiai iš laiškų, ir žodyno lentelė, kurioje kiekvienam laiškui surašomi jame pasitaikiusių žodžių kiekiai

In [ ]:
zodynas = []

for tekstas in treniravimo_duomenys["Tekstas"]:
    for zodis in tekstas:
        if not zodis.isnumeric():
            zodynas.append(zodis)

zodynas = list(set(zodynas))

In [ ]:
zodziu_sk_tekste = {unikalus_zodis: [0]*len(treniravimo_duomenys["Tekstas"]) for unikalus_zodis in zodynas}

for indeksas, tekstas in enumerate(treniravimo_duomenys["Tekstas"]):
    for zodis in tekstas:
        zodziu_sk_tekste[zodis][indeksas] += 1

In [ ]:
zodziu_lentele = pd.DataFrame(zodziu_sk_tekste)
zodziu_lentele.head()
#zodziu_sk_tekste

In [ ]:
treniravimo_rinkinys = pd.concat([treniravimo_duomenys, zodziu_lentele], axis = 1)
treniravimo_rinkinys.head()

<h1>4. Išankstiniai skaičiavimai.</h1>

Kaikuriuos modeliams reikalingus skaičiavimus galima atlikti iš anksto

<h2>Rankiniu būdu (vykdant kodą praleisti šitą dalį, ji yra demonstracijai).</h2>

In [ ]:
#atrenkami tikri ir šlamšto laiškai
geri_laiskai = treniravimo_rinkinys[treniravimo_rinkinys["Tipas"] == 'ham']
slamsto_laiskai = treniravimo_rinkinys[treniravimo_rinkinys["Tipas"] == 'spam']

#P(tikras) ir P(šlamstas)
p_tikras = len(geri_laiskai) / len(treniravimo_rinkinys)
p_slamstas = len(slamsto_laiskai) / len(treniravimo_rinkinys)

#Žodžių tikruose laiškuose kiekis
tikru_laisku_ilgiai = geri_laiskai["Tekstas"].apply(len)
geru_zodziu_sk = tikru_laisku_ilgiai.sum()

#Žodžių šlamšto laiškuose kiekis
slamsto_laisku_ilgiai = slamsto_laiskai["Tekstas"].apply(len)
slamsto_zodziu_sk = slamsto_laisku_ilgiai.sum()

#Žodyno ilgis (unikalių žodžių skaičius)
unikaliu_sk = len(zodynas)

#Žodžių pasirodymo tikimybės
p_zodis_ = [sum(zodziu_lentele[zodis]) for zodis in zodziu_lentele]
p_zodis = {zodis:sum(zodziu_lentele[zodis]/sum(p_zodis_)) for zodis in zodziu_lentele}    

In [ ]:
tikri_zodziai = {unikalus_zodis:0 for unikalus_zodis in zodynas}
slamsto_zodziai = {unikalus_zodis:0 for unikalus_zodis in zodynas}

for zodis in zodynas:
    zodzio_sk_kai_tikras = geri_laiskai[zodis].sum()
    p_zodzio_kai_tikras = (zodzio_sk_kai_tikras + 1) / (geru_zodziu_sk + unikaliu_sk)
    #p_zodzio_kai_tikras = (zodzio_sk_kai_tikras) / (geru_zodziu_sk)
    tikri_zodziai[zodis] = p_zodzio_kai_tikras
    
    zodzio_sk_kai_slamstas = slamsto_laiskai[zodis].sum()
    p_zodzio_kai_slamstas = (zodzio_sk_kai_slamstas + 1) / (slamsto_zodziu_sk + unikaliu_sk)
    #p_zodzio_kai_slamstas = (zodzio_sk_kai_slamstas) / (slamsto_zodziu_sk)
    slamsto_zodziai[zodis] = p_zodzio_kai_slamstas

<h2>Programiškai, su sklearn funkcijomis (naudoti šitą dalį).</h2>

In [ ]:
X = treniravimo_duomenys
X["Tekstas"] = apdoroti_tekstus(X["Tekstas"])
X = X["Tekstas"].str.join(sep = " ")
y = treniravimo_duomenys["Tipas"]

X_ = testavimo_duomenys
X_["Tekstas"] = apdoroti_tekstus(X_["Tekstas"])
X_ = X_["Tekstas"].str.join(sep = " ")
y_zinomi = testavimo_duomenys["Tipas"]

In [ ]:
count_vec = CountVectorizer()
zodziu_maisas = count_vec.fit_transform(X)
zodziu_maisas = np.array(zodziu_maisas.todense())
X = zodziu_maisas
zodziu_maisas_ = count_vec.transform(X_)
zodziu_maisas_ = np.array(zodziu_maisas_.todense())
X_ = zodziu_maisas_

<h1>5. Preliminarūs modeliai.</h1>
Su pagal nutylėjimą naudojamais parametrais

<h2>5.1. Bajeso variantas.</h2>

<h3>5.1.1 Bajeso rankinis variantas (vykdant kodą praleisti šitą skyrių, jis yra demonstracijai).</h3>

In [ ]:
#klasifikavimo funkcija Bajesui 2
def klasifikuoti_Bajesu_2(laiskas):
    laiskas = apdoroti_teksta(laiskas)
    
    p_tikras_kai_tekstas = p_tikras
    p_slamstas_kai_tekstas = p_slamstas

    for zodis in tekstas:
        #print(zodis)
        if zodis in tikri_zodziai:
            p_tikras_kai_tekstas *= tikri_zodziai[zodis]
            #print(p_tikras_kai_tekstas)

        if zodis in slamsto_zodziai:
            p_slamstas_kai_tekstas *= slamsto_zodziai[zodis]
            #print(p_slamstas_kai_tekstas)
    
    if p_tikras_kai_tekstas > p_slamstas_kai_tekstas:
        return 'ham'
    elif p_tikras_kai_tekstas < p_slamstas_kai_tekstas:
        return 'spam'
    else:
        return 'unknown'

In [ ]:
#testavimas
B2_test = testavimo_duomenys
B2_test["Klasifikavimas"] = B2_test["Tekstas"].apply(klasifikuoti_Bajesu_2)
B2_test.head()

In [ ]:
B2_test["Klasifikavimas"].value_counts(normalize = False)

In [ ]:
B2_test["Tekstas"].head(1).apply(klasifikuoti_Bajesu_2)

In [ ]:
teisingai = 0
visi = B2_test.shape[0]

for laiskas in testavimo_duomenys.iterrows():
    laiskas_ = laiskas[1]
    if laiskas_["Tipas"] == laiskas_["Klasifikavimas"]:
        teisingai += 1

print(teisingai/visi)

<h3>5.1.2 Bajeso Scikit-Learn MultinomialNB variantas (su tik dvejom klasem).</h3>

In [ ]:
SK_Bayes = MultinomialNB(force_alpha = True)
SK_Bayes.fit(X, y)

In [ ]:
y_bayes = SK_Bayes.predict(X_)

In [ ]:
n = 0
l = len(y_bayes)

for i in range(len(y_bayes)):
    if y_zinomi[i] == y_bayes[i]:
        n += 1

print(n/l)
print(metrics.accuracy_score(y_zinomi, y_bayes))
print(metrics.f1_score(y_zinomi, y_bayes, average = "macro"))

<h2>5.2 kNN Scikit-Learn variantas.</h2>

In [ ]:
SK_knn = KNeighborsClassifier()
SK_knn.fit(X, y)

In [ ]:
y_knn = SK_knn.predict(X_)

In [ ]:
n = 0
l = len(y_knn)

for i in range(len(y_knn)):
    if y_zinomi[i] == y_knn[i]:
        n += 1

print(n/l)
print(metrics.accuracy_score(y_zinomi, y_knn))
print(metrics.f1_score(y_zinomi, y_knn, average = "macro"))

<h2>5.3 SVC Scikit-Learn variantas.</h2>

In [ ]:
SK_svc = SVC(random_state = 333)
SK_svc.fit(X, y)

In [ ]:
y_svc = SK_svc.predict(X_)

In [ ]:
n = 0
l = len(y_svc)

for i in range(len(y_svc)):
    if y_zinomi[i] == y_svc[i]:
        n += 1

print(n/l)
print(metrics.accuracy_score(y_zinomi, y_svc))
print(metrics.f1_score(y_zinomi, y_svc, average = "macro"))

<h1>6. Modeliu optimizavimas.</h1>

Optimizuojami modeliai

<h2>6.1. Bajeso optimizavimas.</h2>

<h3>Pirmas raundas</h3>

In [ ]:
#keičiama aplha (išlyginimo konstanta)
alpha = [(i+1)/100 for i in range(100)]
bayes_rez = {}
for i in alpha:
    SK_Bayes_ = MultinomialNB(alpha = i, force_alpha = True)
    scores = cross_val_score(SK_Bayes_, X, y)
    bayes_rez[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("bajesas baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = bayes_rez, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

<h3>Galutinis MultinomialNB</h3>

In [ ]:
SK_Bayes_ = MultinomialNB(alpha = 0.77, force_alpha = True) #su optimaliausiu parametru
SK_Bayes_.fit(X, y)
y_bayes_ = SK_Bayes_.predict(X_)
metrics.accuracy_score(y_zinomi, y_bayes_)

<h2>6.2. KNN optimizavimas.</h2>

<h3>Pirmas raundas</h3>

In [ ]:
#keičiamas n_neighbors (vertinimo kaimynų skaičius)
n_neighbors = [i+1 for i in range(10)]
knn_rez_par1 = {}
for i in n_neighbors:
    SK_knn_ = KNeighborsClassifier(n_neighbors = i)
    scores = cross_val_score(SK_knn_, X, y)
    knn_rez_par1[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("knn n_neighbors baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = knn_rez_par1, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas weights (kaimynų svoriai)
weights = ['uniform', 'distance']
knn_rez_par2 = {}
for i in weights:
    SK_knn_ = KNeighborsClassifier(weights = i)
    scores = cross_val_score(SK_knn_, X, y)
    knn_rez_par2[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("knn weights baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = knn_rez_par2, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas algorithm (kaimynų paieškos algoritmai)
algorithm = ['auto', 'ball_tree', 'kd_tree', 'brute']
knn_rez_par3 = {}
for i in algorithm:
    SK_knn_ = KNeighborsClassifier(algorithm = i)
    scores = cross_val_score(SK_knn_, X, y)
    knn_rez_par3[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("knn algorithm baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = knn_rez_par3, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas metric (atstumų apskaičiavimo algoritmai)
metric = ['cityblock', 'cosine', 'euclidean', 'haversine', 'l1', 'l2', 'manhattan', 'nan_euclidean']
knn_rez_par4 = {}
for i in metric:
    SK_knn_ = KNeighborsClassifier(metric = i)
    scores = cross_val_score(SK_knn_, X, y)
    knn_rez_par4[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("knn metric baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = knn_rez_par4, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

<h3>Antras raundas</h3>

In [ ]:
#keičiamas n_neighbors (vertinimo kaimynų skaičius)
n_neighbors = [i+1 for i in range(10)]
knn_rez_par1 = {}
for i in n_neighbors:
    SK_knn_ = KNeighborsClassifier(n_neighbors = i, metric = 'cosine')
    scores = cross_val_score(SK_knn_, X, y)
    knn_rez_par1[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("knn n_neighbors baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = knn_rez_par1, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas weights (kaimynų svoriai)
weights = ['uniform', 'distance']
knn_rez_par2 = {}
for i in weights:
    SK_knn_ = KNeighborsClassifier(weights = i, metric = 'cosine')
    scores = cross_val_score(SK_knn_, X, y)
    knn_rez_par2[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("knn weights baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = knn_rez_par2, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas algorithm (kaimynų paieškos algoritmai)
algorithm = ['auto', 'brute']
knn_rez_par3 = {}
for i in algorithm:
    SK_knn_ = KNeighborsClassifier(algorithm = i, metric = 'cosine')
    scores = cross_val_score(SK_knn_, X, y)
    knn_rez_par3[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("knn algorithm baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = knn_rez_par3, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

<h3>Trečias raundas</h3>

In [ ]:
#keičiamas weights (kaimynų svoriai)
weights = ['uniform', 'distance']
knn_rez_par2 = {}
for i in weights:
    SK_knn_ = KNeighborsClassifier(n_neighbors = 1, weights = i, metric = 'cosine')
    scores = cross_val_score(SK_knn_, X, y)
    knn_rez_par2[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("knn weights baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = knn_rez_par2, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas algorithm (kaimynų paieškos algoritmai)
algorithm = ['auto', 'brute']
knn_rez_par3 = {}
for i in algorithm:
    SK_knn_ = KNeighborsClassifier(n_neighbors = 1, algorithm = i, metric = 'cosine')
    scores = cross_val_score(SK_knn_, X, y)
    knn_rez_par3[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("knn algorithm baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = knn_rez_par3, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

<h3>Ketvirtas raundas</h3>

In [ ]:
#keičiamas algorithm (kaimynų paieškos algoritmai)
knn_rez_par23 = {}
for i in algorithm:
    for j in weights:
        SK_knn_ = KNeighborsClassifier(n_neighbors = 1, algorithm = i, weights = j, metric = 'cosine')
        scores = cross_val_score(SK_knn_, X, y)
        knn_rez_par23[i + "+" + j] = scores.mean()
print("knn baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = knn_rez_par23, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

<h3>Galutinis kNN</h3>

In [ ]:
SK_knn_ = KNeighborsClassifier(n_neighbors = 1, algorithm = 'brute', weights = 'distance', metric = 'cosine')
SK_knn_.fit(X, y)
y_knn_ = SK_knn_.predict(X_)
metrics.accuracy_score(y_zinomi, y_knn_)

<h2>6.3. SVC optimizavimas.</h2>

<h3>Pirmas raundas</h3>

In [ ]:
#keičiamas c (išlyginimo koeficientas) LABAI ILGAI GALVOJA, 0.01-1 vis dideja, reikia kitokio intervalo?
c = [1.3+(i+1)/100 for i in range(40)]
svc_rez_par1 = {}
for i in c:
    SK_svc_ = SVC(C = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y, cv = 2)
    svc_rez_par1[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc C baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par1, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas kernel (funkcijos branduolys)
kernel = ['linear', 'poly', 'rbf', 'sigmoid']
svc_rez_par2 = {}
for i in kernel:
    SK_svc_ = SVC(kernel = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y)
    svc_rez_par2[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc kernel baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par2, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas shrinking (suspaudimas?)
shrinking = [True, False]
svc_rez_par3 = {}
for i in shrinking:
    SK_svc_ = SVC(shrinking = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y)
    svc_rez_par3[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc shrinking baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par3, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas probability (ar apskaičiuoti tikimybes (gali būti lėtas))
probability = [True, False]
svc_rez_par4 = {}
for i in probability:
    SK_svc_ = SVC(probability = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y, cv = 2)
    svc_rez_par4[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc probability baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par4, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas tol (mastymo slenkstis)
tol = [(i+1)/1000 for i in range(100)]
svc_rez_par5 = {}
for i in tol:
    SK_svc_ = SVC(tol = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y, cv = 2)
    svc_rez_par5[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc tol baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par5, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas class_weight (klasių svoriai)
class_weight = [None, 'balanced']
svc_rez_par7 = {}
for i in class_weight:
    SK_svc_ = SVC(class_weight = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y)
    svc_rez_par7[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc class_weight baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par7, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas max_iter (maksimalus paieškos iteracijų skaičius (-1 – neribota))
#max_iter = [1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000, 6500, 7000, 7500, -1]
max_iter = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, -1]
svc_rez_par8 = {}
for i in max_iter:
    SK_svc_ = SVC(max_iter = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y, cv = 2)
    svc_rez_par8[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc max_iter baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par8, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas decision_function_shape (palyginimo forma)
decision_function_shape = ['ovo', 'ovr']
svc_rez_par9 = {}
for i in decision_function_shape:
    SK_svc_ = SVC(decision_function_shape = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y)
    svc_rez_par9[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc decision_function_shape baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par9, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas break_ties (jei decision_function_shape = 'ovr' ir klasių sk. > 2, ar pamiršti atmestus kandidatus (intensyviai naudoja resursus))
break_ties = [True, False]
svc_rez_par10 = {}
for i in break_ties:
    SK_svc_ = SVC(random_state = 333)
    scores = cross_val_score(SK_svc_, X, y)
    svc_rez_par10[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc break_ties baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par10, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

<h3>Antras raundas</h3>
C = 1,63

In [ ]:
#keičiamas kernel (funkcijos branduolys)
kernel = ['linear', 'poly', 'rbf', 'sigmoid']
svc_rez_par2 = {}
for i in kernel:
    SK_svc_ = SVC(C = 1.63, kernel = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y)
    svc_rez_par2[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc kernel baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par2, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas shrinking (suspaudimas?)
shrinking = [True, False]
svc_rez_par3 = {}
for i in shrinking:
    SK_svc_ = SVC(C = 1.63, shrinking = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y)
    svc_rez_par3[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc shrinking baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par3, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas probability (ar apskaičiuoti tikimybes (gali būti lėtas))
probability = [True, False]
svc_rez_par4 = {}
for i in probability:
    SK_svc_ = SVC(C = 1.63, probability = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y, cv = 2)
    svc_rez_par4[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc probability baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par4, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas tol (mastymo slenkstis)
tol = [(i+1)/1000 for i in range(100)]
svc_rez_par5 = {}
for i in tol:
    SK_svc_ = SVC(C = 1.63, tol = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y, cv = 2)
    svc_rez_par5[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc tol baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par5, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas class_weight (klasių svoriai)
class_weight = [None, 'balanced']
svc_rez_par7 = {}
for i in class_weight:
    SK_svc_ = SVC(C = 1.63, class_weight = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y)
    svc_rez_par7[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc class_weight baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par7, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas max_iter (maksimalus paieškos iteracijų skaičius (-1 – neribota))
#max_iter = [1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000, 6500, 7000, 7500, -1]
max_iter = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, -1]
svc_rez_par8 = {}
for i in max_iter:
    SK_svc_ = SVC(C = 1.63, max_iter = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y, cv = 2)
    svc_rez_par8[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc max_iter baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par8, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas decision_function_shape (palyginimo forma)
decision_function_shape = ['ovo', 'ovr']
svc_rez_par9 = {}
for i in decision_function_shape:
    SK_svc_ = SVC(C = 1.63, decision_function_shape = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y)
    svc_rez_par9[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc decision_function_shape baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par9, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas break_ties (jei decision_function_shape = 'ovr' ir klasių sk. > 2, ar pamiršti atmestus kandidatus (intensyviai naudoja resursus))
break_ties = [True, False]
svc_rez_par10 = {}
for i in break_ties:
    SK_svc_ = SVC(C = 1.63, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y)
    svc_rez_par10[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc break_ties baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par10, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

<h3>Trečias raundas</h3>
C = 1,63
tol = 0,008

In [ ]:
#keičiamas kernel (funkcijos branduolys)
kernel = ['linear', 'poly', 'rbf', 'sigmoid']
svc_rez_par2 = {}
for i in kernel:
    SK_svc_ = SVC(C = 1.63, tol = 0.008, kernel = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y)
    svc_rez_par2[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc kernel baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par2, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas shrinking (suspaudimas?)
shrinking = [True, False]
svc_rez_par3 = {}
for i in shrinking:
    SK_svc_ = SVC(C = 1.63, tol = 0.008, shrinking = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y)
    svc_rez_par3[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc shrinking baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par3, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas probability (ar apskaičiuoti tikimybes (gali būti lėtas))
probability = [True, False]
svc_rez_par4 = {}
for i in probability:
    SK_svc_ = SVC(C = 1.63, tol = 0.008, probability = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y, cv = 2)
    svc_rez_par4[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc probability baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par4, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas class_weight (klasių svoriai)
class_weight = [None, 'balanced']
svc_rez_par7 = {}
for i in class_weight:
    SK_svc_ = SVC(C = 1.63, tol = 0.008, class_weight = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y)
    svc_rez_par7[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc class_weight baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par7, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas max_iter (maksimalus paieškos iteracijų skaičius (-1 – neribota))
#max_iter = [1500, 2000, 2500, 3000, 3500, 4000, 4500, 5000, 5500, 6000, 6500, 7000, 7500, -1]
max_iter = [100, 200, 300, 400, 500, 600, 700, 800, 900, 1000, 1100, 1200, 1300, 1400, 1500, -1]
svc_rez_par8 = {}
for i in max_iter:
    SK_svc_ = SVC(C = 1.63, tol = 0.008, max_iter = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y, cv = 2)
    svc_rez_par8[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc max_iter baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par8, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas decision_function_shape (palyginimo forma)
decision_function_shape = ['ovo', 'ovr']
svc_rez_par9 = {}
for i in decision_function_shape:
    SK_svc_ = SVC(C = 1.63, tol = 0.008, decision_function_shape = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y)
    svc_rez_par9[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc decision_function_shape baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par9, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas break_ties (jei decision_function_shape = 'ovr' ir klasių sk. > 2, ar pamiršti atmestus kandidatus (intensyviai naudoja resursus))
break_ties = [True, False]
svc_rez_par10 = {}
for i in break_ties:
    SK_svc_ = SVC(C = 1.63, tol = 0.008, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y)
    svc_rez_par10[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc break_ties baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par10, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

<h3>Ketvirtas raundas</h3>
C = 1,63
tol = 0,008
max_iter = -1

In [ ]:
#keičiamas kernel (funkcijos branduolys)
kernel = ['linear', 'poly', 'rbf', 'sigmoid']
svc_rez_par2 = {}
for i in kernel:
    SK_svc_ = SVC(C = 1.63, tol = 0.008, max_iter = -1, kernel = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y)
    svc_rez_par2[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc kernel baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par2, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas shrinking (suspaudimas?)
shrinking = [True, False]
svc_rez_par3 = {}
for i in shrinking:
    SK_svc_ = SVC(C = 1.63, tol = 0.008, max_iter = -1, shrinking = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y)
    svc_rez_par3[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc shrinking baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par3, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas probability (ar apskaičiuoti tikimybes (gali būti lėtas))
probability = [True, False]
svc_rez_par4 = {}
for i in probability:
    SK_svc_ = SVC(C = 1.63, tol = 0.008, max_iter = -1, probability = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y, cv = 2)
    svc_rez_par4[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc probability baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par4, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas class_weight (klasių svoriai)
class_weight = [None, 'balanced']
svc_rez_par7 = {}
for i in class_weight:
    SK_svc_ = SVC(C = 1.63, tol = 0.008, max_iter = -1, class_weight = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y)
    svc_rez_par7[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc class_weight baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par7, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas decision_function_shape (palyginimo forma)
decision_function_shape = ['ovo', 'ovr']
svc_rez_par9 = {}
for i in decision_function_shape:
    SK_svc_ = SVC(C = 1.63, tol = 0.008, max_iter = -1, decision_function_shape = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y)
    svc_rez_par9[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc decision_function_shape baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par9, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas break_ties (jei decision_function_shape = 'ovr' ir klasių sk. > 2, ar pamiršti atmestus kandidatus (intensyviai naudoja resursus))
break_ties = [True, False]
svc_rez_par10 = {}
for i in break_ties:
    SK_svc_ = SVC(C = 1.63, tol = 0.008, max_iter = -1, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y)
    svc_rez_par10[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc break_ties baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par10, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

<h3>Penktas raundas</h3>
C = 1,63
tol = 0,008
max_iter = -1
kernel = 'rbf'

In [ ]:
#keičiamas shrinking (suspaudimas?)
shrinking = [True, False]
svc_rez_par3 = {}
for i in shrinking:
    SK_svc_ = SVC(C = 1.63, tol = 0.008, max_iter = -1, kernel = 'rbf', shrinking = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y)
    svc_rez_par3[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc shrinking baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par3, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas probability (ar apskaičiuoti tikimybes (gali būti lėtas))
probability = [True, False]
svc_rez_par4 = {}
for i in probability:
    SK_svc_ = SVC(C = 1.63, tol = 0.008, max_iter = -1, kernel = 'rbf', probability = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y, cv = 2)
    svc_rez_par4[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc probability baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par4, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas class_weight (klasių svoriai)
class_weight = [None, 'balanced']
svc_rez_par7 = {}
for i in class_weight:
    SK_svc_ = SVC(C = 1.63, tol = 0.008, max_iter = -1, kernel = 'rbf', class_weight = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y)
    svc_rez_par7[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc class_weight baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par7, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas decision_function_shape (palyginimo forma)
decision_function_shape = ['ovo', 'ovr']
svc_rez_par9 = {}
for i in decision_function_shape:
    SK_svc_ = SVC(C = 1.63, tol = 0.008, max_iter = -1, kernel = 'rbf', decision_function_shape = i, random_state = 333)
    scores = cross_val_score(SK_svc_, X, y)
    svc_rez_par9[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc decision_function_shape baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par9, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

In [ ]:
#keičiamas break_ties (jei decision_function_shape = 'ovr' ir klasių sk. > 2, ar pamiršti atmestus kandidatus (intensyviai naudoja resursus))
break_ties = [True, False]
svc_rez_par10 = {}
for i in break_ties:
    SK_svc_ = SVC(C = 1.63, tol = 0.008, max_iter = -1, kernel = 'rbf', random_state = 333)
    scores = cross_val_score(SK_svc_, X, y)
    svc_rez_par10[i] = scores.mean()
    print(str(i) + ": " + str(scores.mean()))
print("svc break_ties baigta")

In [ ]:
tmp = pd.DataFrame.from_dict(data = svc_rez_par10, orient = 'index')
tmp.loc[tmp[0] == tmp[0].max()]

<h3>Galutinis SVC</h3>

In [ ]:
SK_svc_ = SVC(C = 1.63, tol = 0.008, max_iter = -1, kernel = 'rbf', class_weight = None, random_state = 333)
SK_svc_.fit(X, y)
y_svc_ = SK_svc_.predict(X_)
metrics.accuracy_score(y_zinomi, y_svc_)